In [ ]:
import h5py
import pickle as pkl
import numpy as np
import os
import sys

In [ ]:
if sys.platform == 'win32':
    dataset_dir = 'D:/Users/mickey/Data/datasets/ecog/goose_wireless'
elif sys.platform == 'linux':
    dataset_dir = '/home/ws5/manolan/data/datasets/ecog/goose_wireless'
assert os.path.exists(dataset_dir), f'{dataset_dir} not found!'

In [ ]:
data_suffix = '_fl50u120'
parent_dataset = 'gw_250' + data_suffix
full_dataset_h5f = h5py.File(os.path.join(dataset_dir,parent_dataset),mode='r')
with open(os.path.join(dataset_dir,f'{parent_dataset}_param.pkl'),mode='rb') as pf:
    full_dataset_param_dict = pkl.load(pf)

In [ ]:
# create smaller dataset, randomly sampling from each portion
factor = 5
frac_train_size = full_dataset_h5f['train_ecog'+data_suffix].shape[0]//factor
frac_valid_size = full_dataset_h5f['valid_ecog'+data_suffix].shape[0]//factor
frac_test_size = full_dataset_h5f['test_ecog'+data_suffix].shape[0]//factor
frac_train_idx = np.random.permutation(full_dataset_h5f['train_ecog'+data_suffix].shape[0])[:frac_train_size]
frac_valid_idx = np.random.permutation(full_dataset_h5f['valid_ecog'+data_suffix].shape[0])[:frac_valid_size]
frac_test_idx = np.random.permutation(full_dataset_h5f['test_ecog'+data_suffix].shape[0])[:frac_test_size]

In [ ]:
# save dataset and parameter pickle (same as the full set, I didn't actually save the sample order in the original).
frac_dataset_h5f = h5py.File(os.path.join(dataset_dir,f'gw_250'+data_suffix+f'_dec{factor}'),mode='w')
frac_dataset_h5f['train_ecog'+data_suffix] = full_dataset_h5f['train_ecog'+data_suffix].value[frac_train_idx,:,:]
frac_dataset_h5f['valid_ecog'+data_suffix] = full_dataset_h5f['valid_ecog'+data_suffix].value[frac_valid_idx,:,:]
frac_dataset_h5f['test_ecog'+data_suffix] = full_dataset_h5f['test_ecog'+data_suffix].value[frac_test_idx,:,:]
frac_dataset_h5f['dt'] = full_dataset_h5f['dt'][()]
with open(os.path.join(dataset_dir,'gw_250'+data_suffix+'_pilot_param.pkl'),mode='wb') as pf:
    pkl.dump(full_dataset_param_dict,pf)

In [ ]:
full_dataset_h5f.close()
frac_dataset_h5f.close()

In [ ]:
print(frac_train_size,frac_valid_size,frac_test_size)